In [13]:
import subprocess
from pydub import AudioSegment
import math


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")


In [17]:
import openai
import glob

# transcript = openai.Audio.transcribe(
#     "whisper-1",
#     open("./files/chunks/chunk_0.mp3", "rb"),
# )

# transcript["text"]

def transcribe_chunks(chunk_folder, destination):
    files = sorted(glob.glob(f"{chunk_folder}/*.mp3"))
    return files

transcribe_chunks("./files/chunks", "...")

['./files/chunks/chunk_0.mp3',
 './files/chunks/chunk_1.mp3',
 './files/chunks/chunk_2.mp3',
 './files/chunks/chunk_3.mp3',
 './files/chunks/chunk_4.mp3',
 './files/chunks/chunk_5.mp3']